In [1]:
import os

In [2]:
%pwd

'/Users/abrar/ML-DS-Projects/Practice/End-to-End-ML-Project-with-MLOps/research'

In [3]:
os.chdir('../')

In [4]:
import sys
sys.path.append("src")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [17]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

# Setup configuration manager
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):   
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [12]:
import os
from urllib import request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        """
        
        Downloads the data from the source URL if it doesn't exist

        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                self.config.source_URL, 
                self.config.local_data_file)
            logger.info(f"{filename} downloaded!\nInfo: {headers}")
        else:
            logger.info(f"Data already exists at {self.config.local_data_file} with size {get_size(self.config.local_data_file)}")
    
    def extract_zip_file(self):
        """
        Extracts the zip file to the specified directory

        """
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
            logger.info(f"Data extracted at {self.config.unzip_dir}")

In [20]:
# Pipeline
try:
    config_mgr = ConfigurationManager()
    data_ingestion_config = config_mgr.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
    logger.info("Data Ingestion pipeline completed successfully!")
except Exception as e:
    logger.error(f"Error in data ingestion pipeline: {str(e)}")

[2025-01-01 21:10:57,622: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-01 21:10:57,660: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-01 21:10:57,663: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-01 21:10:57,668: INFO: common: created directory at: artifacts]
[2025-01-01 21:10:57,697: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-01 21:11:00,621: INFO: 596605324: artifacts/data_ingestion/data.zip downloaded!
Info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A823:EA936:38E2B1C:3DAB5B9:67756913
Accept-Ranges: bytes
Date: Wed, 01 Jan 2025 16:1